In [13]:
import pandas as pd
from tqdm import tqdm
from collections import Counter
import re

# we can not find 'PanGU-Σ', 'Luminous' in close Source index, cannot find 'Galatica', 'YaLM', 'PanGu-α' in open source list , so delete them.
openSourceList = ['T5', 'mT5', 'CPM-2','T0','GPT-NeoX-20B','CodeGen','Tk-Instruct','UL2','OPT','NLLB','BLOOM','GLM','Flan-T5','mT0','BLOOMZ','OPT-IML','Pythia','LLaMA','Vicuna','ChatGLM','CodeGeeX','Koala','GPT-2','GPT2','GPT 2']
closeSourceList = ['GShard','GPT-3','GPT 3', 'GPT3','LaMDA','HyperCLOVA','Codex','ERNIE 3','Jurassic-1','FLAN','MT-NLG','Yuan 1.0','Anthropic','WebGPT','Gopher','ERNIE 3.0 Titan','GLaM','InstructGPT','ChatGPT','AlphaCode','Chinchilla','PaLM','Cohere','AlexaTM','Sparrow','WeLM','U-PaLM','Flan-PaLM','Flan-U-PaLM','Alpaca','GPT-4' ,'GPT 4', 'GPT4','Claude']

readfile = "/data/jx4237data/DataForChatGPTinnovationWaves/"
df = pd.read_csv(readfile + 'LLM09enhanced.csv',dtype=object)
# some paper we need to delete:
# “fake” palm paper 1909.02134
# fake openai 1506.04006
delete_list = ['1506.04006','1909.02134']
df = df[~df['id'].isin(delete_list)]
# model2paper
model2paper = {}
for model in tqdm(openSourceList+closeSourceList):
    if model in model2paper:
        print('error')
    model2paper[model] = set()
    for paper in df[df['abstract'].str.contains("\\b%s\\b"%model) | df['abstract'].str.lower().str.contains("\\b%s\\b"%model.lower())  | df['title'].str.contains("\\b%s\\b"%model) | df['title'].str.lower().str.contains("\\b%s\\b"%model.lower())]['id']  :
        model2paper[model].add(paper)

for i in model2paper['GPT 4']:
    model2paper['GPT-4'].add(i)
for i in model2paper['GPT4']:
    model2paper['GPT-4'].add(i)
for i in model2paper['GPT 3']:
    model2paper['GPT-3'].add(i)
for i in model2paper['GPT3']:
    model2paper['GPT-3'].add(i)
for i in model2paper['GPT2']:
    model2paper['GPT-2'].add(i)
for i in model2paper['GPT 2']:
    model2paper['GPT-2'].add(i)
del model2paper['GPT 4']
del model2paper['GPT4']
del model2paper['GPT3']
del model2paper['GPT 3']
del model2paper['GPT2']
del model2paper['GPT 2']

# paper2model
paper2model = {}
for model, paperSet in model2paper.items():
    for paper in paperSet:
        if paper not in paper2model:
            paper2model[paper] =[model]
        else:
            paper2model[paper].append(model)


openPaperSet = set()
closePaperSet = set()
for k,v in model2paper.items():
    if k in closeSourceList:
        for paper in v:
            closePaperSet.add(paper)
    if k in openSourceList:
        for paper in v:
            openPaperSet.add(paper)
pureOpen = openPaperSet - closePaperSet
pureClose = closePaperSet - openPaperSet
mixed = openPaperSet & closePaperSet
all = openPaperSet | closePaperSet

paper2pubdate = {}
for i in range(len(df)):
    paper2pubdate[list(df['id'])[i]] = list(df['update_date'])[i]

100%|██████████| 59/59 [00:21<00:00,  2.80it/s]


In [14]:
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')  # Download the WordNet dataset if you haven't already
# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /home/jx4237/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
with open('stopwords.txt') as f:
    stopwordslist = f.read().splitlines()
stopwordslist.append('')

In [16]:
def genrawdata(modelname):
    c = Counter()
    for i in df[df['id'].isin(model2paper[modelname])]['abstract'] + df[df['id'].isin(model2paper[modelname])]['title']:
        for word in re.split('[, \.!?:()]+', i.strip()):
            if lemmatizer.lemmatize(word.lower()) in stopwordslist:
                continue
            else:
                c.update([lemmatizer.lemmatize(word.lower())])
    totalunique = len(c)
    sumcounts = sum(c.values())
    model_list = []
    for k,v in sorted(c.items(), key= lambda x:-x[1]):
        model_list.append({
            'types':k,
            'counts':v,
            'totalunique':totalunique,
            'probs':v/sumcounts,
        })
    return {modelname:model_list}

In [17]:
def genrawdataByname(paperset,name):
    c = Counter()
    for i in df[df['id'].isin(paperset)]['abstract'] + df[df['id'].isin(paperset)]['title']:
        for word in re.split('[, \.!?:()]+', i.strip()):
            if lemmatizer.lemmatize(word.lower()) in stopwordslist:
                continue
            else:
                c.update([lemmatizer.lemmatize(word.lower())])
    totalunique = len(c)
    sumcounts = sum(c.values())
    model_list = []
    for k,v in sorted(c.items(), key= lambda x:-x[1]):
        model_list.append({
            'types':k,
            'counts':v,
            'totalunique':totalunique,
            'probs':v/sumcounts,
        })
    return {name:model_list}

In [18]:
import json
openSourceList = ['T5', 'mT5', 'CPM-2','T0','GPT-NeoX-20B','CodeGen','Tk-Instruct','UL2','OPT','NLLB','BLOOM','GLM','Flan-T5','mT0','BLOOMZ','OPT-IML','Pythia','LLaMA','Vicuna','ChatGLM','CodeGeeX','Koala','GPT-2']
closeSourceList = ['GShard','GPT-3','LaMDA','HyperCLOVA','Codex','ERNIE 3','Jurassic-1','FLAN','MT-NLG','Yuan 1.0','Anthropic','WebGPT','Gopher','ERNIE 3.0 Titan','GLaM','InstructGPT','ChatGPT','AlphaCode','Chinchilla','PaLM','Cohere','AlexaTM','Sparrow','WeLM','U-PaLM','Flan-PaLM','Flan-U-PaLM','Alpaca','GPT-4' ,'Claude']

opensorcePaper = set()
closedsorcePaper = set()

for i in openSourceList:
    opensorcePaper = opensorcePaper | model2paper[i]
for i in closeSourceList:
    closedsorcePaper = closedsorcePaper | model2paper[i]



s = json.dumps({**genrawdataByname(opensorcePaper,'open sorce'), **genrawdataByname(closedsorcePaper,'closed sorce'),**genrawdata('LLaMA'),**genrawdata('GPT-4'),**genrawdata('GPT-3'),**genrawdata('T5'), **genrawdata('GPT-2'),**genrawdata('PaLM'), **genrawdata('ChatGPT'), })
open("LLM09enhanced.json","w").write(s)

8185701